# Big Data Machine Learning Classification with Spark

Big Data Churn prediction with Spark
Please use pyspark machine learning package to predict customers' behavior.

from pyspark.ml.classification import GBTClassifier

In [1]:
#pip install pyspark  #No module named 'pyspark'

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("churn.csv")

In [4]:
df

,Unnamed: 0,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Churn
0,0,Cameron Williams,42.0,11066.80,0,7.22,8.0,1
1,1,Kevin Mueller,41.0,11916.22,0,6.50,11.0,1
2,2,Eric Lozano,38.0,12884.75,0,6.67,12.0,1
3,3,Phillip White,42.0,8010.76,0,6.71,10.0,1
4,4,Cynthia Norton,37.0,9191.58,0,5.56,9.0,1
...,...,...,...,...,...,...,...,...
895,895,Paul Miller,42.0,12800.82,1,3.62,8.0,0
896,896,Natalie Hodges,52.0,9893.92,0,6.91,7.0,0
897,897,Ana Smith,45.0,12056.18,0,5.46,4.0,0
898,898,Justin Leonard,51.0,6517.93,1,5.47,10.0,0


In [15]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Create a Spark session
spark = SparkSession.builder.appName("Churn Prediction").getOrCreate()

# Load dataset 
data = spark.read.csv("churn.csv", header=True, inferSchema=True)
# Data preprocessing
# Assuming your dataset has columns like 'features' and 'label' representing features and churn status respectively
# You might need to adjust this part based on your actual dataset columns
feature_columns = [col for col in data.columns if (col!="Names") & (col!="Churn")]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
data = assembler.transform(data).select("features", "Churn")

# Split the data into training and test sets (80% train, 20% test)
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Initialize GBT classifier
gbt = GBTClassifier(labelCol="Churn", featuresCol="features", maxIter=10)

# Train the model
model = gbt.fit(train_data)

# Make predictions on the test data
predictions = model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="Churn")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:", accuracy)

# You can also inspect feature importances if needed
feature_importances = model.featureImportances
print("Feature Importances:", feature_importances)

# Finally, don't forget to stop the Spark session
spark.stop()


Accuracy: 0.9918032786885246
Feature Importances: (6,[0,1,2,3,4,5],[0.9315828367591981,0.01167105289743017,0.0025456851093621983,0.0031358142617962503,0.01242455166132453,0.03864005931088877])
